In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Dense

# Загрузка данных
df = pd.read_csv('имена.csv')

# Предположим, что у нас есть столбцы 'слово1', 'слово2', и 'пол'
words1 = df['Составляющее 1'].tolist()
words2 = df['Составляющее 2'].tolist()
genders = df['Пол'].tolist()
names = df['Имя'].tolist()

# Уникальные значения
unique_words = list(set(words1 + words2))
unique_names = list(set(names))
unique_genders = list(set(genders))

word_to_idx = {word: i for i, word in enumerate(unique_words)}
name_to_idx = {name: i for i, name in enumerate(unique_names)}
gender_to_idx = {gender: i for i, gender in enumerate(unique_genders)}

# Преобразуем слова, пол и имена в индексы
X_word1 = np.array([word_to_idx[w] for w in words1])
X_word2 = np.array([word_to_idx[w] for w in words2])
X_gender = np.array([gender_to_idx[g] for g in genders])
y = np.array([name_to_idx[n] for n in names])

# Гиперпараметры
embedding_dim = 128
hidden_dim = 128
vocab_size = len(unique_words)
name_size = len(unique_names)
gender_size = len(unique_genders)

# Определение модели
word_input1 = Input(shape=(1,), name='word_input1')
word_input2 = Input(shape=(1,), name='word_input2')
gender_input = Input(shape=(1,), name='gender_input')

word_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1)
gender_embedding = Embedding(input_dim=gender_size, output_dim=embedding_dim, input_length=1)

embedded_word1 = word_embedding(word_input1)
embedded_word2 = word_embedding(word_input2)
embedded_gender = gender_embedding(gender_input)

# Объединяем эмбеддинги
merged = Concatenate(axis=1)([embedded_word1, embedded_word2, embedded_gender])
lstm_out = LSTM(hidden_dim)(merged)
output = Dense(name_size, activation='softmax')(lstm_out)

model = Model(inputs=[word_input1, word_input2, gender_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Вывод структуры модели
model.summary()

# Обучение модели
model.fit([X_word1, X_word2, X_gender], y, epochs=50, batch_size=4)

# Генерация имени
def generate_name(word1, word2, gender, model, word_to_idx, gender_to_idx, idx_to_name):
    # Пытаемся достать индексы из слов
    try:
        word1_idx = word_to_idx[word1]
        word2_idx = word_to_idx[word2]
        gender_idx = gender_to_idx[gender]
    except KeyError:
        raise ValueError("Одно из слов или пол не найдены в наборе данных.")

    input_data = [np.array([word1_idx]), np.array([word2_idx]), np.array([gender_idx])]
    pred = model.predict(input_data)
    predicted_name_idx = np.argmax(pred, axis=1)[0]

    return idx_to_name[predicted_name_idx]

idx_to_name = {v: k for k, v in name_to_idx.items()}

# Пример использования
word1 = "Любовь"
word2 = "Мир"
gender = "Мужской"
try:
    generated_name = generate_name(word1, word2, gender, model, word_to_idx, gender_to_idx, idx_to_name)
    print(f'Сгенерированное имя: {generated_name}')
except ValueError as e:
    print(e)